In [22]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

data = pd.read_csv("preprocessed_apartment_rentals_no_state_dummies_nocluster.csv")

# Features und Ziel
X = data.drop(columns=["price", "cityname"])
y = data["price"]

# Numerische Spalten
num_cols = X.select_dtypes(include=["int64", "float64"]).columns

# Skalieren
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])





# Regression

# Ridge Regression

In [23]:
from Nested_CV_targetEncoding import NestedCVRegressorWithTargetEncoding

In [26]:
from sklearn.linear_model import Ridge
import numpy as np
ridge_params = {"alpha": np.logspace(-3, 3, 10)}

ridge_cv = NestedCVRegressorWithTargetEncoding(model = Ridge(), param_grid = ridge_params, encode_cols=["state"])
ridge_cv.run(X, y, output=True)



Fitting 5 folds for each of 10 candidates, totalling 50 fits
Outer Fold 1/5 | Best Params: {'model__alpha': np.float64(215.44346900318823)} | Fit Time: 0.150s | Outer R²: 0.531 | MSE: 221852.736
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Outer Fold 2/5 | Best Params: {'model__alpha': np.float64(46.41588833612773)} | Fit Time: 0.150s | Outer R²: 0.521 | MSE: 222042.034
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Outer Fold 3/5 | Best Params: {'model__alpha': np.float64(215.44346900318823)} | Fit Time: 0.156s | Outer R²: 0.524 | MSE: 221633.741
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Outer Fold 4/5 | Best Params: {'model__alpha': np.float64(215.44346900318823)} | Fit Time: 0.150s | Outer R²: 0.521 | MSE: 222227.988
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Outer Fold 5/5 | Best Params: {'model__alpha': np.float64(46.41588833612773)} | Fit Time: 0.173s | Outer R²: 0.525 | MSE: 220366.437


In [27]:
number_outer_cv_splits = 5
model_evaluation = pd.DataFrame({"Fold": range(1, number_outer_cv_splits+1)})
model_evaluation['Modell'] = ['Ridge'] * number_outer_cv_splits
model_evaluation['R_2'] = ridge_cv.get_r2_scores()
model_evaluation['MSE'] = ridge_cv.get_mse_scores()
model_evaluation["runtime"] = ridge_cv.get_fit_times()
model_evaluation

,Fold,Modell,R_2,MSE,runtime
0,1,Ridge,0.530843,221852.736077,0.150203
1,2,Ridge,0.521225,222042.033796,0.149859
2,3,Ridge,0.523888,221633.740846,0.155940
3,4,Ridge,0.521152,222227.988179,0.149595
4,5,Ridge,0.525471,220366.437025,0.173166


In [28]:
model_evaluation.to_pickle('benchmarking/Ridgeregression_targetencoding.pkl')

# Analyse
Welche Preise werden gut, welche schlechter vorhergesagt?

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# 1️⃣ Best Alpha aus Nested CV
best_parameters = ridge_cv.get_best_params()
alpha_values = [float(d['alpha']) for d in best_parameters]
mean_alpha = np.mean(alpha_values)
print("Final Alpha:", mean_alpha)


# 2️⃣ Daten splitten
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
plt.hist(y_test)
plt.show()
# 3️⃣ Finales Ridge-Modell trainieren
final_model = Ridge(alpha=mean_alpha)
final_model.fit(X_train, y_train)

# 4️⃣ Vorhersagen auf Testdaten
y_pred = final_model.predict(X_test)

# 5️⃣ Scores berechnen
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Test MSE:", mse)
print("Test R²:", r2)

# 6️⃣ Residual Plot: echte Preise vs absolute Fehler
abs_error = y_test - y_pred

plt.figure(figsize=(8,5))
plt.scatter(y_test, abs_error, c= X_test[:,1], alpha=0.5)
scatter = plt.scatter(y_test, abs_error, c=X_test[:,5], alpha=0.5, cmap='viridis')
plt.xlabel("True Prices (y_test)")
plt.ylabel("Absolute Error |y_true - y_pred|")
plt.title("Residual Plot: True Prices vs Prediction Error")
plt.grid(True)
plt.legend()
cbar = plt.colorbar(scatter)
cbar.set_label('Feature X_test[:,5]')
plt.show()



KeyError: 'alpha'